#  Big Data Processing with Spark

In my earlier 3 notebooks, I discussed in detail about how to install Spark fast, uploading data in Colab, dealing with missing data values and selecting fields. 

This notebook focuses on Data Processing and Data Analysis as a way to get useful information from data



## PART 1. Configure PySpark environment

Copy & Paste code below. 

Read more https://github.com/kyramichel/Spark/blob/master/PySpark_GoogleColab.ipynb

In [1]:
#update the packages existing on the machine
!apt-get update

#install java 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


#install spark: get the file
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
    
#unzip the file
!tar xf spark-2.4.1-bin-hadoop2.7.tgz

#set up the ennvironmental variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.2-bin-hadoop2.7"

#install finspark  
!pip install -q findspark

#importing findspark adds pyspark to the system path, so that next time you can import pyspark like any other python library
import findspark
findspark.init("/content/spark-2.4.1-bin-hadoop2.7")

import pyspark

#SparkContext: the entry point of spark functionality is the interface to running a spark cluster manager
from pyspark import SparkContext, SparkConf


#import a spark session
from pyspark.sql import SparkSession
#create a session
spark = SparkSession.builder.getOrCreate()
spark

#test the installation
df0 = spark.sql("select 'PySpark' as Hello")
df0.show()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

# PART 2. Data Processing

Let's get deeper in data processing and query data in Spark using both Python and SQL.

First, click on File panel, then upload and select your data file to upload it in Colab. More on how to get data in Colab @ https://github.com/kyramichel/Spark/blob/master/DataPysparkCloudColab.ipynb


In [81]:
#loadd data as df
df = spark.read.csv('data2.csv', header=True, inferSchema=True)
df.show(5)  

+---+--------+-----+-----------------+--------------+-----+-------------+--------+---------+------+
| id| Product|Price|             Name|          City|State|      Country|Latitude|Longitude|US Zip|
+---+--------+-----+-----------------+--------------+-----+-------------+--------+---------+------+
|  1|Product1| 1200|           Betina|     Parkville|   MO|United States|  39.195|-94.68194| 64152|
|  2|Product1| 1200|Federica e Andrea|       Astoria|   OR|United States|46.18806|  -123.83| 97103|
|  3|Product2| 3600|           Gerd W|Cahaba Heights|   AL|United States|33.52056| -86.8025| 35243|
|  4|Product1| 1200|         LAURENCE|     Mickleton|   NJ|United States|   39.79|-75.23806|  8056|
|  5|Product1| 1200|            Fleur|        Peoria|   IL|United States|40.69361|-89.58889| 61601|
+---+--------+-----+-----------------+--------------+-----+-------------+--------+---------+------+
only showing top 5 rows



In [82]:
#check data types
df.dtypes

[('id', 'int'),
 ('Product', 'string'),
 ('Price', 'int'),
 ('Name', 'string'),
 ('City', 'string'),
 ('State', 'string'),
 ('Country', 'string'),
 ('Latitude', 'double'),
 ('Longitude', 'double'),
 ('US Zip', 'int')]

## Eliminat uneeded features & missing values using drop():

More details on drop() and select() at https://github.com/kyramichel/Spark/blob/master/PySpark_DataProcessing1.ipynb


In [61]:
#Eliminate undeeded columns
col_list = ['Name', 'City', "US Zip"]
df1= df.drop(*col_list)

In [62]:
#check columns
df1.columns

['id', 'Product', 'Price', 'State', 'Country', 'Latitude', 'Longitude']

In [63]:
#Eliminate all rows that contain missing values
df2 = df1.na.drop()

In [64]:
#return the number of missing values per column
from pyspark.sql.functions import col, when, count
df2.select(*(count(when(col(c).isNull(), c)).alias(c) for c in df2.columns)).show()


+---+-------+-----+-----+-------+--------+---------+
| id|Product|Price|State|Country|Latitude|Longitude|
+---+-------+-----+-----+-------+--------+---------+
|  0|      0|    0|    0|      0|       0|        0|
+---+-------+-----+-----+-------+--------+---------+



In [65]:
df2.show()

+---+--------+-----+-----+-------------+--------+----------+
| id| Product|Price|State|      Country|Latitude| Longitude|
+---+--------+-----+-----+-------------+--------+----------+
|  1|Product1| 1200|   MO|United States|  39.195| -94.68194|
|  2|Product1| 1200|   OR|United States|46.18806|   -123.83|
|  3|Product2| 3600|   AL|United States|33.52056|  -86.8025|
|  4|Product1| 1200|   NJ|United States|   39.79| -75.23806|
|  5|Product1| 1200|   IL|United States|40.69361| -89.58889|
|  6|Product1| 1200|   TN|United States|36.34333| -88.85028|
|  7|Product1| 1200|   NY|United States|40.71417| -74.00639|
|  8|Product1| 1200|   TX|United States|29.42389| -98.49333|
|  9|Product1| 1200|   ID|United States|43.69556|-116.35306|
| 10|Product1| 1200|   NJ|United States|40.03222| -74.95778|
| 11|Product1| 1200|   UT|United States|40.76083|-111.89028|
| 12|Product1| 1200|   CA|United States|   32.64|-117.08333|
| 13|Product1| 1200|   TX|United States|29.61944| -95.63472|
| 14|Product1| 1200|   N

### Describe data

In [66]:
#summary statistics - mean, std for numeric fields
df2.describe().show()

+-------+------------------+--------+------------------+------+-------------+------------------+-------------------+
|summary|                id| Product|             Price| State|      Country|          Latitude|          Longitude|
+-------+------------------+--------+------------------+------+-------------+------------------+-------------------+
|  count|               989|     989|               989|   989|          989|               989|                989|
|   mean| 497.3346814964611|    null|1635.2881698685542|  null|         null| 39.02358567118307|-41.816108273407494|
| stddev|288.60640692444576|    null|1158.9440271070878|  null|         null|19.557241666334374|  67.34419133543398|
|    min|                 1|Product1|               250|    AK|    Argentina|           -41.465|         -159.48528|
|    max|               998|Product3|             13000|Zurich|United States|          64.83778|        174.7666667|
+-------+------------------+--------+------------------+------+-

## Interpolation using ML

Interpolation allows to imput missing values using other records in the dataset. More details: https://github.com/kyramichel/Spark/blob/master/Data_Cleansing_Pyspark.ipynb

Here I use Spark ML to simply impute the NaN by calling an Imputer:

Imputation strategyies: "mean", "median" and "mode"

- mean imputation strategy replaces missing values using the mean value of the column

- median and  mode are useful for imputing categorical feature values.


In [105]:
#mean imputation 
from pyspark.ml.feature import Imputer
imputer = Imputer(inputCols=["Latitude"],outputCols=["Latitude"])
model = imputer.fit(df2)
df2 = model.transform(df2)

In [109]:
#median imputation
from pyspark.ml.feature import Imputer
imputer = Imputer(inputCols=["Longitude"],outputCols=["Longitude"]).setStrategy("median")
model = imputer.fit(df2)
df2 = model.transform(df2)

In [110]:
df2.show(3)

+---+--------+-----+-----+-------------+--------+---------+
| id| Product|Price|State|      Country|Latitude|Longitude|
+---+--------+-----+-----+-------------+--------+---------+
|  1|Product1| 1200|   MO|United States|  39.195|-94.68194|
|  2|Product1| 1200|   OR|United States|46.18806|  -123.83|
|  3|Product2| 3600|   AL|United States|33.52056| -86.8025|
+---+--------+-----+-----+-------------+--------+---------+
only showing top 3 rows



In [111]:
#Imputer needs col to be imputed be float/double otherwise it will throw a casting error
df2.dtypes

[('id', 'int'),
 ('Product', 'string'),
 ('Price', 'int'),
 ('State', 'string'),
 ('Country', 'string'),
 ('Latitude', 'double'),
 ('Longitude', 'double')]

In [112]:
#first cast Price to float
df3 = df2.withColumn("Price", df2['Price'].cast('float'))
df3.dtypes

[('id', 'int'),
 ('Product', 'string'),
 ('Price', 'float'),
 ('State', 'string'),
 ('Country', 'string'),
 ('Latitude', 'double'),
 ('Longitude', 'double')]

In [116]:
#mode imputation
imputer = Imputer(inputCols=['Price'],outputCols=['Price']).setStrategy("mode")
model = imp.fit(df3)
df3 =model.transform(df3)


In [115]:
imputer = Imputer(inputCols=["Price", "Latitude","Longitude"],outputCols=["Price", "Latitude","Longitude"])
model = imputer.fit(df3)
df3 = model.transform(df3)


*In general, you can use Imputer for all columns/slice = df.columns[a:b] of a df:*


from pyspark.sql.functions import col

df = df.select(*(col(c).cast("float").alias(c) for c in df.columns))

imputer = Imputer(inputCols=df.columns, outputCols=["{}_cleaned".format(c) for c in df.columns])


etc

## Creating a features column

In ML is useful

In [142]:
feature_cols = ['Latitude', 'Longitude'] #omit undeeded columns
print(feature_cols)

['Latitude', 'Longitude']


In [143]:
#import the vector assembler
from pyspark.ml.feature import VectorAssembler

In [147]:
assembler = VectorAssembler(inputCols=feature_cols,outputCol="geo_coor")


In [148]:
#use transform method to transform our dataset
df4 = assembler.transform(df3)  
df4.show(5)

+---+--------+------+-----+-------------+--------+---------+--------------------+
| id| Product| Price|State|      Country|Latitude|Longitude|            geo_coor|
+---+--------+------+-----+-------------+--------+---------+--------------------+
|  1|Product1|1200.0|   MO|United States|  39.195|-94.68194|  [39.195,-94.68194]|
|  2|Product1|1200.0|   OR|United States|46.18806|  -123.83|  [46.18806,-123.83]|
|  3|Product2|3600.0|   AL|United States|33.52056| -86.8025| [33.52056,-86.8025]|
|  4|Product1|1200.0|   NJ|United States|   39.79|-75.23806|   [39.79,-75.23806]|
|  5|Product1|1200.0|   IL|United States|40.69361|-89.58889|[40.69361,-89.58889]|
+---+--------+------+-----+-------------+--------+---------+--------------------+
only showing top 5 rows



### Feature scaling is an important step in ML data preprocessing 

We can use StandardScaler to scalerize the “feature” column


In [151]:
from pyspark.ml.feature import StandardScaler


In [152]:
standardscaler=StandardScaler().setInputCol("geo_coor").setOutputCol(
"Scaled_coor")

df5=standardscaler.fit(df4).transform(df4)


In [155]:
df5.show(3)

+---+--------+------+-----+-------------+--------+---------+-------------------+--------------------+
| id| Product| Price|State|      Country|Latitude|Longitude|           geo_coor|         Scaled_coor|
+---+--------+------+-----+-------------+--------+---------+-------------------+--------------------+
|  1|Product1|1200.0|   MO|United States|  39.195|-94.68194| [39.195,-94.68194]|[2.00411697460740...|
|  2|Product1|1200.0|   OR|United States|46.18806|  -123.83| [46.18806,-123.83]|[2.36168580355110...|
|  3|Product2|3600.0|   AL|United States|33.52056| -86.8025|[33.52056,-86.8025]|[1.71397176411139...|
+---+--------+------+-----+-------------+--------+---------+-------------------+--------------------+
only showing top 3 rows



## Partitioning a dataset: Train-Test Split 

is data preprocessing technique needed in ML 

Ex: 70-30 train-test split

In [156]:
train, test = df4.randomSplit([0.7, 0.3], seed=123)

In [160]:
df5.count()

989

In [158]:
train.count()

701

In [159]:
test.count()

288